# Tutorial #3: Enable recurrent materialization and run batch inference

In this tutorial series you will experience how features seamlessly integrates all the phases of ML lifecycle: Prototyping features, training and operationalizing.

In the previous part of the tutorial you learnt to experiment with features, train the model and register the model along with the feature-retrieval spec. In this tutorial you will learn how to run batch inference for the registered model.

You will perform the following:
- Enable recurrent materialization for the `transactions` feature set
- Run batch inference pipeline on the registered model

# Prerequisites
1. Please ensure you have executed the previous parts of this tutorial series

# Setup

#### Configure Azure ML spark notebook

1. In the "Compute" dropdown in the top nav, select "Serverless Spark Compute". 
1. Click on "configure session" in top status bar -> click on "Python packages" -> click on "upload conda file" -> select the file azureml-examples/sdk/python/featurestore-sample/project/env/conda.yml from your local machine; Also increase the session time out (idle time) if you want to avoid running the prerequisites frequently




#### Start spark session

In [ ]:
# run this cell to start the spark session (any code block will start the session ). This can take around 10 mins.
print("start spark session")

Optional: Try running this if your the Conda yaml file takes too long to load in the spark configuration setting:

In [ ]:
# Uncomment the following line to install the required packages:
# !pip install protobuf==3.19.6 aiohttp==3.8.4 azureml-featurestore[online]==1.1.1 azure-ai-ml==1.24.0 \
#     azure-mgmt-msi azure-mgmt-redis azure-mgmt-authorization==3.0.0 pandas==1.5.3 scikit-learn \
#     azureml-inference-server-http azure-cli

#### Setup root directory for the samples

In [ ]:
import os

# please update the dir to ./Users/<your_user_alias> (or any custom directory you uploaded the samples to).
# You can find the name from the directory structure in the left nav
root_dir = "Users/admin/Azure_MLOps_v3/featurestore_sample"

if os.path.isdir(root_dir):
    print("The folder exists.")
else:
    print("The folder does not exist. Please create or fix the path")

#### Initialize the project workspace CRUD client
This is the current workspace where you will be running the tutorial notebook from

In [ ]:
### Initialize the MLClient of this project workspace
import os
from azure.ai.ml import MLClient
from azure.ai.ml.identity import AzureMLOnBehalfOfCredential

project_ws_sub_id = "..."
project_ws_rg = "..."
project_ws_name = "..."

# connect to the project workspace
ws_client = MLClient(
    AzureMLOnBehalfOfCredential(), project_ws_sub_id, project_ws_rg, project_ws_name
)

#### Initialize the feature store CRUD client
Ensure you update the `featurestore_name` to reflect what you created in part 1 of this tutorial

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.identity import AzureMLOnBehalfOfCredential

# feature store
featurestore_name = (
    "..."  # use the same name from part #1 of the tutorial
)
featurestore_subscription_id = "..."
featurestore_resource_group_name = "..."

# feature store ml client
fs_client = MLClient(
    AzureMLOnBehalfOfCredential(),
    featurestore_subscription_id,
    featurestore_resource_group_name,
    featurestore_name,
)

#### Initialize the feature store core sdk client

In [ ]:
# feature store client
from azureml.featurestore import FeatureStoreClient
from azure.ai.ml.identity import AzureMLOnBehalfOfCredential

featurestore = FeatureStoreClient(
    credential=AzureMLOnBehalfOfCredential(),
    subscription_id=featurestore_subscription_id,
    resource_group_name=featurestore_resource_group_name,
    name=featurestore_name,
)

## Step 1: Enable recurrent materialization on the `transactions` featureset

In part 2 of this tutorial you enabled materialization and performed backfill on the transactions feature set. Backfill is an ondemand one-time operation to compute and store feature values in the materialization store. However when you want to perform inference of the model in production, you might want to keep the materilization store upto date by setting up recurrent materialization jobs. These jobs run on user defined schedule
The recurrent job schedule works in the following way: 
- A window is defined by the interval and frequency. E.g., interval = 3 and frequency = Hour define a 3-hour window
- The first window starts at the start_time defined in the RecurenceTrigger, and so on.
- The first recurrent job will be submitted at the begining of the next window after the update time.
- Later recurrent jobs will be submitted at every window after the first job.

As explained in the previous parts of the tutorials, once data is materialized (backfill/recurrent materialization), feature retrieval will use the materialized data by default.


In [ ]:
from datetime import datetime
from azure.ai.ml.entities import RecurrenceTrigger

transactions_fset_config = fs_client.feature_sets.get(name="transactions", version="1")

# create a schedule that runs the materialization job every 3 hours
transactions_fset_config.materialization_settings.schedule = RecurrenceTrigger(
    interval=3, frequency="Hour", start_time=datetime(2025, 4, 15, 0, 4, 10, 0)
)

fs_poller = fs_client.feature_sets.begin_create_or_update(transactions_fset_config)

print(fs_poller.result())

#### (Optional) Save the feature set asset  yaml with the updated settings

In [ ]:
## uncomment and run
# transactions_fset_config.dump(root_dir + "/featurestore/featuresets/transactions/featureset_asset_offline_enabled_with_schedule.yaml")

#### Track status of the recurrent materialization jobs in the feature store studio UI
This job will every three hours. 

__Action__:

1. Feel free to execute the next step for now (batch inference).
1. In three hours check the recurrent job status via the UI

## Step 2: Run the batch-inference pipeline

In this step you will manually trigger the batch inference pipeline. In a production scenario, this could be trigerred by a ci/cd pipeline based on model registration/approval.

The batch-inference has the following steps:

1. Feature retrieval step: This use the same built-in feature retrieval component that we used in the training pipeline in the part 3 of the tutorial. Incase of training pipeline, we provided feature retreival spec as an input to the component. However in case of batch inference we will pass the registered model as the input and the component will look for feature retrieval spec in the model artifact. Another difference is that in case of training, the observation data had the target variable, however incase of batch inference it will not be present. The feature retrieval step will join the observation data with the features and output the data for batch inference.
1. Batch inference: This step uses the batch inference input data from previous step, runs inference on the model and outputs the data by appending the predicted value.

__Note:__ In this example we use a job for batch inference. You can also use Azure ML's batch endpoints.


Note: Please check batch_inference_pipeline.yaml and make sure the compute cluster is correct. Check these steps:
- model_retrieval_step
- inference_step

In [ ]:
from azure.ai.ml import load_job  # will be used later

# set the batch inference  pipeline path
batch_inference_pipeline_path = (
    root_dir + "/project/fraud_model/pipelines/batch_inference_pipeline.yaml"
)
batch_inference_pipeline_definition = load_job(source=batch_inference_pipeline_path)

# run the training pipeline
batch_inference_pipeline_job = ws_client.jobs.create_or_update(
    batch_inference_pipeline_definition
)

# stream the run logs
ws_client.jobs.stream(batch_inference_pipeline_job.name)

#### Inspect the batch inference output data
1. In the pipeline view, double click on `inference_step` -> in `outputs` card, copy the `Data` field. It will be something like `azureml_995abbc2-3171-461e-8214-c3c5d17ede83_output_data_data_with_prediction:1`. 
1. Paste it in the below cell with name and version separately (notice that the last character is the version, separated by a `:`).
1. You will see the `predict_is_fraud` column generated by the batch inference pipeline

Explanation: Since we did not provide a `name` and `version` in the `outputs` of the `inference_step` in the batch inference pipeline (`/project/fraud_mode/pipelines/batch_inference_pipeline.yaml`), the system created an untracked data asset with a guid as name and version as 1. In the next cell we will be getting the data path from the asset and displaying it.

In [ ]:
inf_data_output = ws_client.data.get(
    name="...",
    version="1",
)
inf_output_df = spark.read.parquet(inf_data_output.path)
display(inf_output_df.head(5))

## Cleanup

Tutorial "5. Develop a feature set with custom source" has instructions for deleting the resources.